# Running Single Stars with POSYDON 

Single stars, hydrogen or helium main sequence stars (HMS; HeMS), can be run with POSYDON specifying the `single_star_grid = True` option in the `ini` file. This function takes care of setting up the MESA simulation to run a single star instead of a binary star system.

You can get the template to run a 0.1Zsun HMS star by running the following code cell. Do not forget to fill up the header of the `ini` file with the appropriate information corresponding to your HPC account. In this example we will run the following MESA inlist branch and commit
```ini
scenario = ['posydon', 'development-c4f90ce2d93595f66751011d2002fc3ab3d090ec', 'HMS-HMS']
```

In [1]:
import os
import shutil
from posydon.config import PATH_TO_POSYDON

path_to_ini = os.path.join(PATH_TO_POSYDON, "grid_params/POSYDON-MESA-INLISTS/r11701/running_scripts/single_HMS_yggdrasil.ini")
shutil.copyfile(path_to_ini, './single_HMS_yggdrasil.ini')

'./single_HMS_yggdrasil.ini'

We now need to generate the `grid.csv` containing 10 initial values for 10 stars, in this case the initial values are just the mass and metallicity paramters. You can do this by running the following code cell. The code that follows was copied from `$PATH_TO_POSYDON/grid_params/POSYDON-MESA-INLISTS/r11701/running_scripts/parameter_space_v2/create_csv.ipynb`.

In [3]:
import os
import csv
import numpy as np
import pandas as pd

def log_range(x_min,x_max,x_n):
    return 10**np.linspace(np.log10(x_min),np.log10(x_max), x_n)

# digit rounding pick 10 to be sure we resolve 10^-4Zsun
NDIG = 10

Zsun = 0.0142
m1_min = 5.
m1_max = 100.
m1_n = 10
m1 = log_range(m1_min,m1_max,m1_n)
met = [0.1*Zsun]

for Z in met:
    with open('./grid.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['initial_z','Zbase','initial_mass'])
        for i in range(m1_n):
            writer.writerow([round(Z,NDIG),round(Z,NDIG),round(m1[i],NDIG)])

In [4]:
!head -n 12 ./grid.csv

initial_z,Zbase,initial_mass
0.00142,0.00142,5.0
0.00142,0.00142,6.9747539698
0.00142,0.00142,9.7294385879
0.00142,0.00142,13.572088083
0.00142,0.00142,18.9323950471
0.00142,0.00142,26.4097595025
0.00142,0.00142,36.8403149864
0.00142,0.00142,51.3904266401
0.00142,0.00142,71.6871164437
0.00142,0.00142,100.0


We are now ready to run the simulation, with the following commands:

```bash
posydon-setup-grid --grid-type fixed --inifile single_HMS_yggdrasil.ini --submission-type slurm
sbatch slurm_job_array_grid_submit.sh
```

Sit back relax, and wait for the grid to finish. Check out the advanced tutorial on how to generate the PSyGrid object for single stars.